Does it work on the KS? Let's check

In [1]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN, Siren
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold
from deepymod.training.sparsity_scheduler import TrainTestPeriodic, Periodic, TrainTest

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

from deepymod.utils.logger import Logger
from deepymod.training.convergence import Convergence
from scipy.io import loadmat
from deepymod.analysis import load_tensorboard

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [2]:
# Prepping data
data = loadmat('kuramoto_sivishinky.mat')

t = data['tt']
x = data['x']
u = data['uu']

# Normalizing data
t = (t - t.min())/(t.max()-t.min()) * 2 - 1
x = (x - x.min())/(x.max()-x.min()) * 2 - 1

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')

# Limiting to non-chaotic part
lower_lim = 80
x_grid = x_grid[:, lower_lim:]
t_grid = t_grid[:, lower_lim:]
u = u[:, lower_lim:]

# %%Making training data
X = np.concatenate((t_grid.reshape(-1, 1), x_grid.reshape(-1, 1)), axis=1)
y = u.reshape(-1, 1)

# Adding noise
noise_level = 0.05
y_noisy = y + noise_level * np.std(y, axis=0) * np.random.randn(*y.shape)
number_of_samples = 25000

# Into tensor
idx = np.random.permutation(y.shape[0])
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32).to(device)
y_train = torch.tensor(y_noisy[idx, :][:number_of_samples], dtype=torch.float32).to(device)

In [25]:
def train_SBL(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          extra_params, 
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    t, a, l = extra_params
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)
    
    M = 10
    N = data_train.shape[0]
    threshold = torch.tensor(1e4).to(data.device)
    
    alpha_threshold = torch.tensor(1e8).to(data.device)
    
    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)
        
        tau_ = torch.exp(t)
        alpha_ = torch.min(torch.exp(a), alpha_threshold)
        lambda_ = torch.min(torch.exp(l), 2 * threshold)
                            
        y = time_derivs[0]
        X = thetas[0] / torch.norm(thetas[0], dim=0, keepdim=True)
        
        p_MSE = N / 2 * (tau_ * torch.mean((prediction - target_train)**2, dim=0) - t + np.log(2*np.pi))
        
        A = torch.diag(lambda_) + alpha_ * X.T @ X
        mn = (lambda_ < threshold)[:, None] * (alpha_ * torch.inverse(A) @ X.T @ y)
        E = alpha_ * torch.sum((y - X @ mn)**2) + mn.T @ torch.diag(lambda_) @ mn
        p_reg = 1/2 * (E + torch.sum(torch.log(torch.diag(A)[lambda_ < threshold])) - (torch.sum(l[lambda_ < threshold]) + N * a) - N * np.log(2*np.pi))

        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
        loss = torch.sum(p_MSE + p_reg)

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE = p_MSE,
                   p_reg = p_reg,
                   tau = tau_,
                   alpha=alpha_,
                  lambda_=lambda_,
                   mn=mn)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            if update_sparsity: 
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)
    

In [26]:
t = torch.nn.Parameter(-torch.log(torch.var(y_train)).to(device))
a = torch.nn.Parameter(-torch.log(torch.var(y_train)).to(device))
l  = torch.nn.Parameter(torch.zeros(10).to(device))

In [27]:
# %%Configuring model
network = Siren(2, [50, 50, 50, 50, 50, 50, 50, 50], 1) # Function approximator
library =  Library1D(poly_order=1, diff_order=4) # Library function
estimator = Threshold(0.1)#PDEFIND(lam=1e-6, dtol=0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

# %% Setting schedulers
sparsity_scheduler = TrainTestPeriodic(patience=8, delta=1e-9)#Periodic(initial_epoch=10000, periodicity=100) # Defining when to apply sparsity
optimizer = torch.optim.Adam([{'params':model.parameters(), 'betas':(0.999, 0.999), 'amsgrad':True, 'lr':0.00025}, {'params':[t, a, l], 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3}]) # Defining optimizer

In [ ]:
train_SBL(model, X_train, y_train, optimizer, [t, a, l], sparsity_scheduler, exp_ID='KS_fast_params', split=0.8, write_iterations=50, max_iterations=50000, delta=0.0, patience=200) 

 26050  MSE: 1.45e+00  Reg: 2.65e-15  L1: 0.00e+00 